# [Failure] Digits with Evolution

In [27]:
import torch
import numpy as np
import sklearn
from sklearn.datasets import load_digits
import sklearn.model_selection

In [2]:
digits = load_digits()

In [3]:
data = digits['data']
targets = digits['target']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(data, targets, test_size=0.28)

In [4]:
X_train, X_test, y_train, y_test = torch.FloatTensor(X_train), torch.FloatTensor(X_test), torch.LongTensor(y_train), torch.LongTensor(y_test)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([1293, 64]),
 torch.Size([504, 64]),
 torch.Size([1293]),
 torch.Size([504]))

In [6]:
from torch import nn
import torch.nn.functional as F

In [7]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        
        return x

In [33]:
model = Classifier()

In [34]:
criterion = torch.nn.CrossEntropyLoss()

In [35]:
def loss(y_pred, y_true):
    return 1/criterion(y_pred, y_true)

In [36]:
def fitness_func(solution):
    nn.utils.vector_to_parameters(solution, model.parameters())
    return loss(model(X_train), y_train) + 0.00000001

In [37]:
def calc_population_fitness(pop):
    fitness = torch.zeros(pop.shape[0])
    for i, state_dict in enumerate(pop):
        fitness[i] = fitness_func(state_dict)
    return fitness

In [38]:
def select_best_parents(pop, num_parents, fitness):
    parents = torch.zeros(num_parents, pop.shape[1])
    topk_fitness =  torch.topk(fitness, num_parents).indices
    for i, parent_n in enumerate(topk_fitness):
        parents[i] = pop[parent_n]
    return parents

In [39]:
def mate(parents, offspring_size):
    offspring = torch.zeros(offspring_size)
    crossover_point = np.random.randint(0, offspring_size[1], size=1)[0]
    for k in range(offspring_size[0]):
        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [61]:
def mutate(offspring, percent_mutate=0.3):
    for k in range(offspring.shape[0]):
#         rand_param = np.random.randint(0, offspring.shape[1], size=1)
        indices = torch.from_numpy(np.random.choice(offspring.shape[1], int(offspring.shape[1]*percent_mutate), replace=False))
        offspring[k, indices] += torch.zeros(indices.shape[0]).uniform_(-6, 6)

## Initialize the initial population

In [62]:
params_shape = nn.utils.parameters_to_vector(model.parameters()).shape

In [63]:
POPULATION_SIZE = 30
NUM_GENERATIONS = 1500
NUM_PARENTS=5

In [64]:
initial_population = nn.init.uniform_(torch.empty(POPULATION_SIZE, *params_shape))

In [65]:
population = initial_population
for generation in range(NUM_GENERATIONS + 1):
    fitness = calc_population_fitness(population)
    best_parents = select_best_parents(population, NUM_PARENTS, fitness)
    offspring = mate(best_parents, (initial_population.shape[0] - best_parents.shape[0], initial_population.shape[1]))
    mutate(offspring, percent_mutate=0.6)
    population = torch.zeros(initial_population.shape)
    population[0:best_parents.shape[0]] = best_parents
    population[best_parents.shape[0]:] = offspring
    if generation % 50 == 0:
        print(f"Generation: {generation}, Best Solution: {best_parents[0]}, Fitness: {fitness_func(best_parents[0])}")

Generation: 0, Best Solution: tensor([0.3041, 0.7955, 0.6211,  ..., 0.0244, 0.3350, 0.7049]), Fitness: 0.42459315061569214
Generation: 50, Best Solution: tensor([ 5.3387, -7.3858,  7.0650,  ...,  1.5879, -1.8055,  3.5905]), Fitness: 0.45253023505210876
Generation: 100, Best Solution: tensor([15.2160, -9.0565,  6.3465,  ...,  1.6760,  2.3495,  5.4337]), Fitness: 0.46289873123168945
Generation: 150, Best Solution: tensor([ 12.5285,  -3.7090,   4.8922,  ...,  -1.6052,   8.4503, -10.8060]), Fitness: 0.46623700857162476
Generation: 200, Best Solution: tensor([12.5285, -3.7090,  4.8922,  ..., -1.3587,  4.3301, -5.8868]), Fitness: 0.4737534821033478
Generation: 250, Best Solution: tensor([12.5285, -3.7090,  4.8922,  ..., -1.3587,  4.3301, -5.8868]), Fitness: 0.4737534821033478
Generation: 300, Best Solution: tensor([12.5285, -3.7090,  4.8922,  ..., -1.3587,  4.3301, -5.8868]), Fitness: 0.4737534821033478
Generation: 350, Best Solution: tensor([ 25.0961,  -5.1450,   8.8294,  ...,  -6.0810,  -2

In [66]:
solution = best_parents[0]
nn.utils.vector_to_parameters(solution, model.parameters())

In [67]:
print("Accuracy: ")
print(((torch.argmax(model(X_test), dim=1) == y_test).sum() / y_test.shape[0]).item() * 100)

Accuracy: 
54.36508059501648


It is at least learning something